In [2]:
# Taller Práctico - Pipeline YOLO + SAM + MiDaS
# Detección, Segmentación y Estimación de Profundidad

# ===========================
# 1. INSTALACIÓN DE DEPENDENCIAS
# ===========================

# Ejecutar estas líneas en Google Colab
!pip install ultralytics
!pip install segment-anything
!pip install opencv-python
!pip install timm
!pip install matplotlib
!pip install torch torchvision
!pip install numpy
!pip install Pillow
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# ===========================
# 2. IMPORTACIÓN DE LIBRERÍAS
# ===========================

import torch
import torchvision.transforms as transforms
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from typing import List, Tuple, Dict
import pandas as pd

# YOLO
from ultralytics import YOLO

# SAM
import sys
sys.path.append('/content/segment-anything')
from segment_anything import sam_model_registry, SamPredictor

# MiDaS
torch.hub.set_dir('/content/torch_hub')

# ===========================
# 3. CONFIGURACIÓN DE MODELOS
# ===========================

class VisionPipeline:
    def __init__(self):
        # Inicializar YOLO
        self.yolo_model = YOLO('yolov8n.pt')  # Modelo nano para velocidad

        # Inicializar SAM
        sam_checkpoint = "/content/sam_vit_h_4b8939.pth"
        model_type = "vit_h"
        self.sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
        self.sam_predictor = SamPredictor(self.sam)

        # Inicializar MiDaS
        self.midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
        self.midas.eval()

        # Transformaciones para MiDaS
        midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
        self.midas_transform = midas_transforms.default_transform

        # Device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.midas.to(self.device)
        self.sam.to(self.device)

        print(f"Modelos inicializados en: {self.device}")

    def detectar_objetos_yolo(self, imagen_path: str, confidence: float = 0.5) -> Dict:
        """
        Detecta objetos usando YOLOv8

        Args:
            imagen_path: Ruta de la imagen
            confidence: Umbral de confianza

        Returns:
            Dict con boxes, clases, confianzas e imagen original
        """
        # Cargar imagen
        imagen = cv2.imread(imagen_path)
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

        # Detección con YOLO
        results = self.yolo_model(imagen_path, conf=confidence)

        boxes = []
        clases = []
        confianzas = []
        nombres_clases = []

        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    # Coordenadas de la caja
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    boxes.append([int(x1), int(y1), int(x2), int(y2)])

                    # Clase y confianza
                    clase_id = int(box.cls[0].cpu().numpy())
                    confianza = float(box.conf[0].cpu().numpy())

                    clases.append(clase_id)
                    confianzas.append(confianza)
                    nombres_clases.append(self.yolo_model.names[clase_id])

        return {
            'imagen': imagen_rgb,
            'boxes': boxes,
            'clases': clases,
            'confianzas': confianzas,
            'nombres_clases': nombres_clases
        }

    def segmentar_con_sam(self, imagen: np.ndarray, boxes: List[List[int]]) -> List[np.ndarray]:
        """
        Segmenta objetos usando SAM con bounding boxes como prompts

        Args:
            imagen: Imagen en formato RGB
            boxes: Lista de bounding boxes [x1, y1, x2, y2]

        Returns:
            Lista de máscaras binarias
        """
        # Configurar imagen en SAM predictor
        self.sam_predictor.set_image(imagen)

        masks = []

        for box in boxes:
            # Convertir box a formato SAM
            input_box = np.array(box)

            # Predecir máscara
            mask, _, _ = self.sam_predictor.predict(
                point_coords=None,
                point_labels=None,
                box=input_box[None, :],
                multimask_output=False,
            )

            masks.append(mask[0])

        return masks

    def estimar_profundidad_midas(self, imagen: np.ndarray) -> np.ndarray:
        """
        Estima profundidad usando MiDaS

        Args:
            imagen: Imagen en formato RGB

        Returns:
            Mapa de profundidad normalizado
        """
        # Transformar imagen
        input_batch = self.midas_transform(imagen).to(self.device)

        # Predecir profundidad
        with torch.no_grad():
            prediction = self.midas(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=imagen.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        # Convertir a numpy y normalizar
        depth_map = prediction.cpu().numpy()
        depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

        return depth_map

    def visualizar_resultados(self, imagen: np.ndarray, boxes: List, masks: List,
                            depth_map: np.ndarray, nombres_clases: List,
                            confianzas: List) -> None:
        """
        Visualiza todos los resultados en una figura combinada
        """
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Pipeline YOLO + SAM + MiDaS - Resultados', fontsize=16)

        # 1. Imagen original con detecciones YOLO
        axes[0, 0].imshow(imagen)
        axes[0, 0].set_title('Detecciones YOLO')
        axes[0, 0].axis('off')

        for i, (box, nombre, conf) in enumerate(zip(boxes, nombres_clases, confianzas)):
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                               fill=False, color='red', linewidth=2)
            axes[0, 0].add_patch(rect)
            axes[0, 0].text(x1, y1-10, f'{nombre}: {conf:.2f}',
                          color='red', fontsize=10, weight='bold')

        # 2. Imagen con máscaras SAM
        axes[0, 1].imshow(imagen)
        axes[0, 1].set_title('Segmentación SAM')
        axes[0, 1].axis('off')

        colors = plt.cm.Set3(np.linspace(0, 1, len(masks)))
        for mask, color in zip(masks, colors):
            axes[0, 1].imshow(mask, alpha=0.5, cmap='jet')

        # 3. Mapa de profundidad MiDaS
        depth_display = axes[0, 2].imshow(depth_map, cmap='plasma')
        axes[0, 2].set_title('Mapa de Profundidad MiDaS')
        axes[0, 2].axis('off')
        plt.colorbar(depth_display, ax=axes[0, 2], fraction=0.046)

        # 4. Combinación: Imagen + Máscaras + Profundidad
        axes[1, 0].imshow(imagen)
        for i, mask in enumerate(masks):
            masked_depth = np.where(mask, depth_map, np.nan)
            axes[1, 0].imshow(masked_depth, alpha=0.6, cmap='viridis')
        axes[1, 0].set_title('Profundidad por Objeto')
        axes[1, 0].axis('off')

        # 5. Recortes de objetos
        if len(masks) > 0:
            # Mostrar primer objeto recortado
            first_mask = masks[0]
            objeto_recortado = imagen.copy()
            objeto_recortado[~first_mask] = [255, 255, 255]  # Fondo blanco
            axes[1, 1].imshow(objeto_recortado)
            axes[1, 1].set_title(f'Objeto: {nombres_clases[0]}')
            axes[1, 1].axis('off')

        # 6. Estadísticas de profundidad
        axes[1, 2].axis('off')
        stats_text = "Estadísticas de Profundidad por Objeto:\n\n"
        for i, (mask, nombre) in enumerate(zip(masks, nombres_clases)):
            depth_obj = depth_map[mask]
            if len(depth_obj) > 0:
                stats_text += f"{nombre} #{i+1}:\n"
                stats_text += f"  Profundidad media: {depth_obj.mean():.3f}\n"
                stats_text += f"  Profundidad min: {depth_obj.min():.3f}\n"
                stats_text += f"  Profundidad max: {depth_obj.max():.3f}\n\n"

        axes[1, 2].text(0.1, 0.9, stats_text, fontsize=10,
                       verticalalignment='top', transform=axes[1, 2].transAxes)
        axes[1, 2].set_title('Análisis Cuantitativo')

        plt.tight_layout()
        plt.show()

    def analizar_segmentaciones_con_profundidad(self, masks: List, nombres_clases: List,
                                               depth_map: np.ndarray, boxes: List) -> pd.DataFrame:
        """
        Análisis cuantitativo de segmentaciones con profundidad

        Returns:
            DataFrame con estadísticas por objeto
        """
        datos = []

        for i, (mask, nombre, box) in enumerate(zip(masks, nombres_clases, boxes)):
            x1, y1, x2, y2 = box
            depth_obj = depth_map[mask]

            if len(depth_obj) > 0:
                datos.append({
                    'objeto_id': i + 1,
                    'clase': nombre,
                    'bbox_x1': x1,
                    'bbox_y1': y1,
                    'bbox_x2': x2,
                    'bbox_y2': y2,
                    'area_pixels': np.sum(mask),
                    'profundidad_media': depth_obj.mean(),
                    'profundidad_std': depth_obj.std(),
                    'profundidad_min': depth_obj.min(),
                    'profundidad_max': depth_obj.max(),
                    'distancia_relativa': 1 - depth_obj.mean()  # Invertir para distancia
                })

        df = pd.DataFrame(datos)
        return df.sort_values('distancia_relativa', ascending=True)

    def aplicar_efecto_bokeh(self, imagen: np.ndarray, masks: List,
                           depth_map: np.ndarray, blur_strength: int = 15) -> np.ndarray:
        """
        Aplica efecto bokeh basado en profundidad
        """
        # Crear máscara combinada de todos los objetos
        mask_objetos = np.zeros(imagen.shape[:2], dtype=bool)
        for mask in masks:
            mask_objetos |= mask

        # Aplicar desenfoque al fondo
        imagen_blur = cv2.GaussianBlur(imagen, (blur_strength, blur_strength), 0)

        # Combinar imagen original (objetos) con fondo desenfocado
        resultado = imagen_blur.copy()
        resultado[mask_objetos] = imagen[mask_objetos]

        return resultado

    def procesar_imagen_completa(self, imagen_path: str, guardar_resultados: bool = True) -> Dict:
        """
        Pipeline completo: YOLO + SAM + MiDaS
        """
        print("🔍 Iniciando detección con YOLO...")
        detecciones = self.detectar_objetos_yolo(imagen_path)

        if not detecciones['boxes']:
            print("❌ No se detectaron objetos en la imagen")
            return None

        print(f"✅ Detectados {len(detecciones['boxes'])} objetos: {detecciones['nombres_clases']}")

        print("🎯 Segmentando con SAM...")
        masks = self.segmentar_con_sam(detecciones['imagen'], detecciones['boxes'])

        print("🌊 Estimando profundidad con MiDaS...")
        depth_map = self.estimar_profundidad_midas(detecciones['imagen'])

        print("📊 Generando visualizaciones...")
        self.visualizar_resultados(
            detecciones['imagen'],
            detecciones['boxes'],
            masks,
            depth_map,
            detecciones['nombres_clases'],
            detecciones['confianzas']
        )

        print("📈 Análisis cuantitativo...")
        df_analisis = self.analizar_segmentaciones_con_profundidad(
            masks, detecciones['nombres_clases'], depth_map, detecciones['boxes']
        )
        print("\nEstadísticas por objeto:")
        print(df_analisis.to_string(index=False))

        # Efecto creativo: Bokeh
        print("🎨 Aplicando efecto bokeh...")
        imagen_bokeh = self.aplicar_efecto_bokeh(detecciones['imagen'], masks, depth_map)

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(detecciones['imagen'])
        plt.title('Imagen Original')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(depth_map, cmap='plasma')
        plt.title('Mapa de Profundidad')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(imagen_bokeh)
        plt.title('Efecto Bokeh')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        if guardar_resultados:
            self.guardar_resultados(imagen_path, detecciones, masks, depth_map, df_analisis)

        return {
            'detecciones': detecciones,
            'masks': masks,
            'depth_map': depth_map,
            'analisis': df_analisis,
            'imagen_bokeh': imagen_bokeh
        }

    def guardar_resultados(self, imagen_path: str, detecciones: Dict,
                          masks: List, depth_map: np.ndarray, df_analisis: pd.DataFrame):
        """
        Guarda resultados en archivos
        """
        base_name = os.path.splitext(os.path.basename(imagen_path))[0]

        # Crear directorios
        os.makedirs('outputs/depth_maps', exist_ok=True)
        os.makedirs('outputs/recortes', exist_ok=True)
        os.makedirs('outputs/segmentaciones', exist_ok=True)

        # Guardar mapa de profundidad
        plt.imsave(f'outputs/depth_maps/{base_name}_depth.png', depth_map, cmap='plasma')

        # Guardar recortes individuales
        for i, (mask, nombre) in enumerate(zip(masks, detecciones['nombres_clases'])):
            objeto_recortado = detecciones['imagen'].copy()
            objeto_recortado[~mask] = [255, 255, 255]
            plt.imsave(f'outputs/recortes/{base_name}_{nombre}_{i}.png', objeto_recortado)

        # Guardar análisis CSV
        df_analisis.to_csv(f'outputs/{base_name}_analisis.csv', index=False)

        print(f"💾 Resultados guardados en directorio 'outputs/'")

# ===========================
# 4. EJEMPLO DE USO
# ===========================

# Inicializar pipeline
pipeline = VisionPipeline()

# Procesar una imagen (subir imagen a Colab primero)
# imagen_path = "/content/mi_imagen.jpg"  # Cambiar por tu imagen
# resultados = pipeline.procesar_imagen_completa(imagen_path)

# ===========================
# 5. FUNCIONES ADICIONALES
# ===========================

def demo_con_imagen_ejemplo():
    """
    Función demo que descarga una imagen de ejemplo y la procesa
    """
    # Descargar imagen de ejemplo
    import urllib.request

    print("📥 Descargando imagen de ejemplo...")
    url = "https://images.unsplash.com/photo-1544465544-da2c851aba8e?ixlib=rb-4.0.3&w=800"
    urllib.request.urlretrieve(url, "ejemplo_street.jpg")

    # Procesar
    pipeline = VisionPipeline()
    resultados = pipeline.procesar_imagen_completa("ejemplo_street.jpg")

    return resultados

def procesar_multiples_imagenes(directorio_imagenes: str):
    """
    Procesa múltiples imágenes en lote
    """
    pipeline = VisionPipeline()

    for archivo in os.listdir(directorio_imagenes):
        if archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            imagen_path = os.path.join(directorio_imagenes, archivo)
            print(f"\n{'='*50}")
            print(f"Procesando: {archivo}")
            print(f"{'='*50}")

            try:
                pipeline.procesar_imagen_completa(imagen_path, guardar_resultados=True)
            except Exception as e:
                print(f"❌ Error procesando {archivo}: {e}")

# Ejemplo de uso:
# resultados = demo_con_imagen_ejemplo()

print("🚀 Pipeline YOLO + SAM + MiDaS listo para usar!")
print("📝 Para procesar una imagen, usa: pipeline.procesar_imagen_completa('ruta_imagen.jpg')")
print("🧪 Para demo con imagen de ejemplo, usa: demo_con_imagen_ejemplo()")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

100%|██████████| 6.25M/6.25M [00:00<00:00, 54.0MB/s]
/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /content/torch_hub/master.zip
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {

Loading weights:  None


Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /content/torch_hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /content/torch_hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100%|██████████| 340M/340M [00:02<00:00, 146MB/s]
Downloading: "https://github.com/isl-org/MiDaS/releases/download/v2_1/midas_v21_384.pt" to /content/torch_hub/checkpoints/midas_v21_384.pt
100%|██████████| 403M/403M [00:09<00:00, 44.1MB/s]


Modelos inicializados en: cpu
🚀 Pipeline YOLO + SAM + MiDaS listo para usar!
📝 Para procesar una imagen, usa: pipeline.procesar_imagen_completa('ruta_imagen.jpg')
🧪 Para demo con imagen de ejemplo, usa: demo_con_imagen_ejemplo()


Using cache found in /content/torch_hub/intel-isl_MiDaS_master
